# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
# from utils import get_openai_api_key, get_serper_api_key

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = ""

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  PDFSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./resume.pdf')
semantic_search_resume = PDFSearchTool(pdf='./resume.pdf')

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [5]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [27]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [28]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market"
         "You return the experience in years based on the profile as well.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
        "You understand the profile and can assess the experience from the profile." 
    )
)

In [29]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market."
         "You should make sure that the resume has achiever kind of words with quantifyable results done."
        "Highlighting the achievemets of the work done, rather than just saying the work has been done.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
        "You have to make sure you're looking to change from a doer "
        "to an acheiever kind of words."
        
    )
)

In [17]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [18]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [19]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [20]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [21]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [22]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [23]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [24]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

- Dislplay the generated `tailored_resume.md` file.

In [25]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

**Noah's Updated Resume:**

---

**Noah Devadatta Bharadwaj**

**Email:** rjdevbharadwajb@gmail.com  
**Portfolio:** amplyfiedai.com  
**Mobile:** +1-782-233-8008  
**Github:** github.com/rajathbharadwaj  

---

**Summary:**

Result-driven Full Stack Engineer with proven experience in developing scalable software solutions. Adept in both front-end and back-end development, with a strong foundation in various programming languages and frameworks. Excellent communication skills with the ability to bridge the gap between technical and business teams. A self-starter comfortable working in early-stage environments with strong project management and organizational skills.

---

**Work Experience:**

**Graduate Assistant**  
*University of Windsor*  
*Sept 2022 - Present*  
- Taught Masters in Applied Computing (MAC) students Advanced Computing Concepts using interactive Canva slides.

**DL Fellow**  
*Remote, Fellowship.ai*  
*Sept 2021 - Jan 2022*  
- Built an Athlete/Fitness coach bot using RASA, integrated Blenderbot 2.0 for handling fallbacks.
- Utilized Semantic Document Search (SDS) for fallback scenarios.

**AI Solution Architect**  
*In-person, BrainGrid Technologies*  
*Nov 2021 - Sept 2022*  
- Developed models on the TensorRT framework, reducing inference time loads and increasing efficiency by 3%.
- Conducted workshops for clients on NVIDIA’s DGX systems and NGC Containers, leading to positive feedback and a 1% business conversion rate.
- Mentored clients on performing model inference using the Jetson Nano for edge use cases.

---

**Projects:**

- **Automated Trading** *(DL, Reinforcement Learning, Selenium, Sentiment Analysis, Forecasting)*  
  Developed software to trade stocks & options market end-to-end without human intervention, achieving 79% accuracy in trades.  
  *Tech:* Python, Streamlit, Stable Baseline-3, OpenAI - Gym, Selenium, TensorFlow, FBProphet  
  *Nov 2021 - Present*

- **DL-based Gaming** *(Deep Learning, Computer Vision)*  
  Architected a DL model to analyze game frames and predict the next move with 85% accuracy.  
  *Tech:* Python, TensorFlow, OpenCV

- **Action Recognition Tagging** *(Deep Learning, Computer Vision)*  
  Recognizes activities in frames, achieving 82% accuracy and a 2% increase in business revenue.  
  *Tech:* Python, TensorFlow, OpenCV

- **Conversational AI** *(Chatbot, NLP, Blenderbot)*  
  Developed a chatbot for sports and fitness queries, using pre-programmed questions.  
  *Tech:* RASA, Blenderbot 2.0

---

**Education:**

**Master of Science in Computer Science - AI Specialization**  
*University of Windsor, Windsor, ON*  
*Sep 2022 - Present*  
*Courses:* Intro to AI, Statistical Learning, Topics in AI, Neural Networks and Deep Learning

**Bachelor of Engineering in Computer Science**  
*KS School of Engineering (KSSEM), Bangalore, India*  
*Aug 2017 - Aug 2021*  
*GPA:* 8.51  
*Courses:* Operating Systems, Data Structures, Analysis Of Algorithms, Artificial Intelligence, Networking, Databases, Java

---

**Skills:**

- **Languages:** Python, Java
- **Frameworks:** SpaCy, Streamlit, TensorFlow, PyTorch/Lightning, Django, TensorRT, Huggingface, Weights & Biases
- **Tools:** Kubernetes, Docker, GIT, NVIDIA NGC
- **Platforms:** Linux, Web, NVIDIA Jetson, Raspberry Pi, GCP
- **Soft Skills:** Leadership, Event Management, Writing, Public Speaking

---

**Characteristics:**

- **Accountability:** Willingness to accept responsibility with integrity.
- **Grit:** Ability to persevere through challenges and perform well under pressure.
- **Scrappy:** Proactive and creative in getting things done with limited resources.
- **Ownership Orientation:** Extreme ownership over all aspects of the product and results-driven nature.

---

**Equal Employment Opportunity:**

AI Fund is committed to providing an environment of mutual respect where equal employment opportunities are available to all applicants without regard to race, color, religion, sex, pregnancy (including childbirth, lactation, and related medical conditions), national origin, age, physical and mental disability, marital status, sexual orientation, gender identity, gender expression, genetic information (including characteristics and testing), military and veteran status, and any other characteristic protected by applicable law. AI Fund believes that diversity and inclusion among our employees is critical to our success as a company, and we seek to recruit, develop, and retain the most talented people from a diverse candidate pool. Selection for employment is decided on the basis of qualifications, merit, and business need.

---

This updated resume highlights Noah's qualifications and experiences, making them relevant to the Full Stack Engineer position at AI Fund.

- Dislplay the generated `interview_materials.md` file.

In [26]:
display(Markdown("./interview_materials.md"))

The following document contains key questions and talking points that Noah should prepare for the initial interview for the Full Stack Engineer position at AI Fund:

---

**Interview Preparation Document for Noah Devadatta Bharadwaj**

### General Questions:
1. **Tell us about yourself and your background in software development.**
   - **Talking Point**: Highlight education, work experience, and key projects that demonstrate a strong foundation in full stack development and AI.

### Technical Skills:
2. **Can you describe your experience with front-end technologies and frameworks?**
   - **Talking Point**: Discuss knowledge of HTML/CSS, JavaScript, and frameworks like Angular and React. Mention specific projects where these skills were applied.

3. **What back-end technologies and databases are you proficient in?**
   - **Talking Point**: Highlight experience with Python, Java, Node.js, MySQL, and MongoDB. Discuss relevant projects and responsibilities.

4. **How have you implemented and managed APIs in your past projects?**
   - **Talking Point**: Provide examples from work experience or projects, such as the Automated Trading system or the Athlete/Fitness coach bot.

### Project Experience:
5. **Can you walk us through the Automated Trading project you developed?**
   - **Talking Point**: Explain the project's objectives, technologies used (Python, TensorFlow, Selenium), and the outcomes achieved (79% accuracy in trades).

6. **How did you approach developing the DL-based Gaming project?**
   - **Talking Point**: Discuss the architecture of the DL model, the use of TensorFlow and OpenCV, and the accuracy achieved.

7. **Tell us more about the Action Recognition Tagging project and its impact.**
   - **Talking Point**: Describe the technology stack (Python, TensorFlow, OpenCV), the business impact (2% revenue increase), and the accuracy of the model.

### Role-Specific Questions:
8. **How do you ensure the scalability and efficiency of the software solutions you develop?**
   - **Talking Point**: Mention the use of TensorRT for improving model efficiency and experience with Kubernetes and Docker for scalability.

9. **Describe a challenging debugging or troubleshooting experience you had and how you resolved it.**
   - **Talking Point**: Provide a specific example, such as troubleshooting fallbacks in the athlete/fitness coach bot using Blenderbot 2.0.

10. **How do you stay updated with the latest technologies and best practices in full stack development?**
    - **Talking Point**: Discuss continuous learning through courses, workshops, and staying active in relevant communities (e.g., GitHub).

### Soft Skills and Characteristics:
11. **Can you provide an example of a time when you demonstrated strong project management skills?**
    - **Talking Point**: Highlight experiences where project management was crucial, such as leading workshops at BrainGrid Technologies.

12. **How do you handle working in a fast-paced and resource-constrained environment?**
    - **Talking Point**: Illustrate with examples that showcase your scrappiness and grit, such as developing the Automated Trading system with limited resources.

13. **What motivates you to take ownership of your projects and drive them to success?**
    - **Talking Point**: Emphasize your ownership orientation and results-driven nature with specific examples from past projects and roles.

---

Noah should use these questions and talking points to prepare for his interview, ensuring he highlights his relevant skills, experiences, and characteristics that align with the Full Stack Engineer position at AI Fund.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)

In [2]:
!ls

L7_job_application_crew.ipynb  fake_resume.md  utils.py


In [10]:
!cat utils.py

# Add your utilities or helper functions to this file.

import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key

def get_serper_api_key():
    load_env()
    openai_api_key = os.getenv("SERPER_API_KEY")
    return openai_api_key


# break line every 80 characters if line is longer than 80 characters
# don't break in the middle of a word
def pretty_print_result(result):
  parsed_result = []
  for line in